In [ ]:
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import requests
import pandas as pd
import numpy as np
import re
import string
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl

In [ ]:
#Automatically grab the daily data required for the Three Factor Model

original_data = []

url = urllib.request.urlopen("http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip")

with ZipFile(BytesIO(url.read())) as my_zip_file:
    for contained_file in my_zip_file.namelist():
        for line in my_zip_file.open(contained_file).readlines():
            original_data.append(line)

In [ ]:
#Cleaning the raw data

modified_data = original_data[4:]

for i in range(len(modified_data)):
    modified_data[i] = modified_data[i].decode('utf-8')
    modified_data[i] = modified_data[i].replace(' ', '')
    modified_data[i] = modified_data[i].replace('\r\n', '')

In [ ]:
#Cleaning the raw data continued

modified_data[0] = 'Date' + modified_data[0]
data_for_df = []

for i in modified_data:
    x = i.split(',')
    data_for_df.append(x)

In [ ]:
df = pd.DataFrame(data_for_df[1:], columns=data_for_df[0])

In [ ]:
df.drop(df.tail(2).index, 
        inplace = True)

df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

df.set_index('Date', inplace=True)

df = df.astype(float)
df = df / 100

In [ ]:
tickers = input("""What stocks are in your portfolio?\n
                Enter each ticker uppercase and seperated by one space\n
                Ex. AAPL MSFT FB\n
                ---> """)

In [ ]:
dates = input("""Would you like to specify the dates for the Fama-French Calculation?\n
                Format is 'YYYY-MM-DD'\n
                EX. 2018-01-01 2019-01-01\n
                Enter 'N/A' to use defualt data range (last 3 years)\n
                Please note: The stocks in your portfolio need to be trading for the dates included\n
                If a stock in the portfolio you entered wasn't publically traded for the full date range\n
                you provide, the program will not run.\n
                ---> """)

In [ ]:
dates = dates.split(' ')

In [ ]:
#use YFinance to grab stock price data for given portfolio and date range

full_stock_data = yf.download(tickers, start=dates[0], end=dates[1])

In [ ]:
close_stock_data = full_stock_data['Adj Close']

close_stock_returns = close_stock_data.pct_change()

In [ ]:
col_names = list(close_stock_data)
stock_weights = []

In [ ]:
#Gather weights for reach stock in the portfolio

weights_q = input("""Is the portfolio equally weighted?\n
                Enter Y/N\n
                ---> """)
if weights_q == 'Y':
    x = 1 / stock_returns.shape[1]
    for i in range(stock_returns.shape[1]):
        stock_weights.append(x)
else:
    for i in col_names:
        weight = float(input(f"""Enter weight in decimal format:
                    {i} ---> """))
        stock_weights.append(weight)

In [ ]:
#Find weighed average returns needed for the model

weighted_close_stock_returns = close_stock_returns * stock_weights

In [ ]:
weighted_close_stock_returns['port_returns'] = weighted_close_stock_returns.sum(axis=1)

In [ ]:
#Merge the weighted portfolio returns with the Fama-French data

FF_final_data = pd.merge(df,weighted_close_stock_returns['port_returns'],
                        how='inner',
                        left_index=True,
                        right_index=True)

In [ ]:
FF_final_data['port_excess_ret'] = FF_final_data['port_returns'] - FF_final_data['RF']

In [ ]:
FF_final_data.head(25)

In [ ]:
#CAPM Model Estimation

X = FF_final_data['Mkt-RF']
Y = FF_final_data['port_excess_ret']
X = sm.add_constant(X)
CAPM = sm.OLS(Y, X).fit()
predictions = CAPM.predict(X) 

print_model = CAPM.summary()
print(print_model)

graph = sns.regplot(x=FF_final_data["Mkt-RF"], y=FF_final_data["port_excess_ret"], data=FF_final_data,
           scatter_kws={'alpha':0.3}, line_kws={'color':'red', 'alpha':0.7}, truncate=True)
graph.set_xticks(pl.frange(round(FF_final_data["Mkt-RF"].min(), 2), round(FF_final_data["Mkt-RF"].max(), 2), 0.01))
graph.set_xlabel('Market Risk factor')
graph.set_ylabel('Portfolio Excess Returns')
graph.set_title('CAPM Regression Graph')

In [ ]:
#Fama-French 3-Factor Model Estimation
X_FF = FF_final_data[['Mkt-RF', 'SMB', 'HML']]
Y_FF = FF_final_data['port_excess_ret']
X_FF = sm.add_constant(X_FF)
Fama_French_Model = sm.OLS(Y_FF, X_FF).fit()
predictions = Fama_French_Model.predict(X_FF) 

print_model = Fama_French_Model.summary()
print(print_model)

graph2 = sns.pairplot(FF_final_data, x_vars=["Mkt-RF", "SMB", "HML"], y_vars=["port_excess_ret"],
             height=5, aspect=.8, kind="reg", plot_kws={'line_kws':{'color':'red', 'alpha':0.7}, 'scatter_kws': {'alpha': 0.1}})

In [ ]:
"""Uncomment out the code below to export the data to an excel file
    The file will be saved in your current working directory"""

#FF_final_data.to_excel('FFModel.xlsx')